In [23]:
from netCDF4 import Dataset
import numpy as np
import os, pathlib

i_path = '/home/inhye_yoo/edu/datasets/'
i_name = 'ERA.sst_anom.195001_201912.nc'
f= Dataset(i_path+i_name, 'r')
dat = f.variables['p'][:,:,:]
f.close()

print(dat.shape)
dat = np.array(dat)
dat = dat.reshape(-1,73,144)
dat = np.where(dat ==-9.99e+08, np.nan, dat)

lat = np.arange(-90.0, 92.5, 2.5)
lat = np.cos(np.radians(lat))
lat = np.full_like(dat, lat[np.newaxis, ..., np.newaxis])
lat = np.where(np.isnan(dat)==True, np.nan, lat)

pick_dat = dat[:, 34:39, 76:97]
pick_lat = lat[:, 34:39, 76:97]

idx = np.nansum(pick_dat*pick_lat, axis=(1,2))/np.nansum(lat, axis=(1,2))

idx = idx.reshape(-1)
idx = idx.reshape(-1,12)

nino34 = np.append(idx[:-1], idx[1:], axis=1)

djf_nino34 = np.mean(nino34[:,11:14],axis=1)
djf_nino34 = djf_nino34.reshape(-1)

o_path = '/home/inhye_yoo/edu/datasets/'
o_name = 'DJF_NINO34_idx_1950_2019'

# save
djf_nino34 = np.array(djf_nino34)
djf_nino34.astype('float32').tofile(o_path+o_name+'.gdat')

ctl = open(o_path+o_name+'.ctl','w')
ctl.write('dset ^'+o_name+'.gdat\n')
ctl.write('undef    -9.99e+08\n')
ctl.write('xdef 1   linear  0.  2.5\n')
ctl.write('ydef 1   linear  -90.    2.5\n')
ctl.write('zdef 1   linear  1   1\n')
ctl.write('tdef '+str(len(djf_nino34))+'    linear  jan1950 1mo\n')
ctl.write('vars 1\n')
ctl.write('nino34   1   1   variable\n')
ctl.write('ENDVARS\n')
ctl.close()

os.system('cdo -f nc import_binary '+o_path+o_name+'.ctl '+o_path+o_name+'.nc')
os.system('rm -f '+o_path+o_name+'.ctl'+o_path+o_name+'.gdat')

(840, 1, 73, 144)


cdo import_binary: Processed 1 variable [0.00s 486MB]


0

In [ ]:
import numpy as np
import matplotllib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import sys
from matplotlib.patches import Rectanglle